TODO: Small description for notebook

## <span style="color:black">Mount drive to notebook</span>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
!pip install --upgrade google-api-python-client
!pip install gensim


## <span style="color:black">Install needed packages</span>

In [2]:
!pip install tld
!pip install pandas==1.0.5
import tld
from tld import get_tld
import nltk
import gensim
import bz2
import json
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import glob

# Add constants/paths
_DATASETS_PATHS = '/content/drive/Shareddrives/ADA LUNATICS 2021/datasets/Quotebank'
_KEYWORDS = 'sexual harassment|me too|' 

  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.3
    Uninstalling pandas-1.3.3:
      Successfully uninstalled pandas-1.3.3


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\willi\\anaconda3\\Lib\\site-packages\\~-ndas\\_libs\\algos.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



ModuleNotFoundError: No module named 'gensim'

### <span style="color:Blue">1) Load Datasets</span>

In [4]:
# Save the datasets paths
quote_datasets = sorted(glob.glob(_DATASETS_PATHS+'/*.bz2'))
quote_datasets

['/content/drive/Shareddrives/ADA LUNATICS 2021/datasets/Quotebank/quotes-2015.json.bz2',
 '/content/drive/Shareddrives/ADA LUNATICS 2021/datasets/Quotebank/quotes-2016.json.bz2',
 '/content/drive/Shareddrives/ADA LUNATICS 2021/datasets/Quotebank/quotes-2017.json.bz2',
 '/content/drive/Shareddrives/ADA LUNATICS 2021/datasets/Quotebank/quotes-2018.json.bz2',
 '/content/drive/Shareddrives/ADA LUNATICS 2021/datasets/Quotebank/quotes-2019.json.bz2',
 '/content/drive/Shareddrives/ADA LUNATICS 2021/datasets/Quotebank/quotes-2020.json.bz2']

#### <span style="color:red">In order to deal with extremely large data we decided chunking the data and apply all the processing and data wrangling steps on chunks then combine the chunk results (inspired by this [article](https://towardsdatascience.com/3-simple-ways-to-handle-large-data-with-pandas-d9164a3c02c1))</span>

In [8]:
from gensim import corpora
corpuses = []
def process_chunk(chunk):
  print(f'Processing chunk with {len(chunk)} rows')
  print(chunk.columns)
  # Remove Phase and Num of occurences columns (not useful and redundant, respectively) to save memory
  chunk.drop(['phase', 'numOccurrences'], axis=1)
  quotes = chunk['quotation'].apply(process_text)
  # quotes = [process_text(quote) for quote in quotes]
  dictionary = corpora.Dictionary(quotes)
  corpus = [dictionary.doc2bow(quote) for quote in quotes]
  model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=15)
  return model

df_reader = pd.read_json(quote_datasets[5], lines=True, compression='bz2', chunksize=1000000)
for chunk in df_reader:
    model = process_chunk(chunk)
    break

Processing chunk with 1000000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')


In [10]:
from gensim import models


topics = model.print_topics(num_words=3)
for topic in topics:
    print(topic)

(0, '0.010*"new" + 0.009*"work" + 0.008*"need"')
(1, '0.009*"state" + 0.009*"peopl" + 0.008*"govern"')
(2, '0.019*"go" + 0.016*"get" + 0.014*"like"')


### <span style="color:Blue">2) Topic Analysis</span>

#### <span style="color:green">A- Manual Extraction Method</span>

In [ ]:
def manual_extraction(df, keywords):
  #CHECK THE QUOTES THAT INCLUDE SPECIFIC KEYWORDS
  quotations = df['quotation']
  results = quotations.str.contains(keywords, regex=False)
  indices = list(results[results == True].index)
  return df.iloc[indices,:]

Add Plots and mention limitations for this method

#### <span style="color:green">B- Parsing URLs Method</span>

In [32]:
def parse_title(url):
    '''Doc'''
    res = get_tld(url, as_object=True)
    return res.parsed_url.path.split('/')[1]


In [33]:
url = 'http://www.santacruzsentinel.com/environment-and-nature/20180630/dan-haifley-our-ocean-backyard-beach-cleanups-fight-plastic-pollution'
parse_title(url)

'environment-and-nature'

Add Plots and mention limitations for this method

#### <span style="color:green">C- NLP-based Method</span>

In [7]:
import re
import nltk

# We need this dataset in order to use the tokenizer
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Also download the list of stopwords to filter out
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def process_text(text):
    # Make all the strings lowercase and remove non alphabetic characters
    text = re.sub('[^A-Za-z]', ' ', text.lower())

    # Tokenize the text; this is, separate every sentence into a list of words
    # Since the text is already split into sentences you don't have to call sent_tokenize
    tokenized_text = word_tokenize(text)

    # Remove the stopwords and stem each word to its root
    clean_text = [
        stemmer.stem(word) for word in tokenized_text
        if word not in stopwords.words('english')
    ]

    # Remember, this final output is a list of words
    return clean_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### <span style="color:Blue">3) Dataset Augmentation</span>

### <span style="color:Blue">4) Correlation/Significance Analysis</span>

In [31]:
with bz2.open(quote_datasets[5], 'rb') as s_file:
  # target url
  for instance in s_file:
    instance = json.loads(instance) # loading a sample
    urls = instance['urls'] # extracting list of links
    # print(urls)
    domains = []
    for url in urls:
        tld = parse_title(url)
        domains.append(tld)
        print(tld.path.split('/')[1])
  
# # displaying the title
# print("Title of the website is : ")
# print (soup.title.get_text())

Streaming output truncated to the last 5000 lines.
featured
daily-dose
2020
2020
world
latest
latest
charleston_scene
ci
news
cricketnext
news
sports
sports
cricket
news
sport
articles
news
Daily
story
story
story
entertainment
budget
business
business-economy
news
members-of-congress-urge-attorney-general-to-release-as-many-prisoners-as-possible_3293052.html
intelligencer
news
news
items
story
news
news
news
lifestyle
news
news
baton_rouge
article
daily-news
news
~
rihanna-new-song-partynextdoor-believe-it-feature-reactions-2020-3
news
jp-morgan-economists-climate-crisis-2645218658.html
category
newyddion
national
2020
delaware-rep-blunt-rochester-joins-biden-s-campaign-team
news
1752186
articles
india-news
headlines
155008
celebrity-news
entertainment
stories
oarticle
entertainment
world
2020
international
world
world
news
school-voucher-scotus-case-isnt-about-choice-its-about-coerced-support-for-religion
2020
story
from-the-magazine
en_us
en_us
1795206
news
news
opinion
in-win-for-e

KeyboardInterrupt: ignored